In [2]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "../../finetuned_models/safe_model/safe_finetuned_model/", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Unsloth 2024.9.post2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [4]:
def getAnswer_completion(promptvals):
    messages=[ {"role": "system", "content": "You are a helpful assistant who responds as shortly as possible. Your responses are only 1-3 words long."}]
    messages.append({"role": "user", "content": promptvals})
    terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    outputs = model.generate(
            input_ids,
            eos_token_id=terminators,
            temperature=0.01,
            output_logits=True,
             return_dict_in_generate=True,
        max_new_tokens=512
        )
    response = outputs[0][0][input_ids.shape[-1]:]
    output=tokenizer.decode(response, skip_special_tokens=True)
    logits = outputs.logits
    return logits,output

In [5]:
import pandas as pd
dataset=pd.read_json("../dataset/triviaqa/triviaqa_1000.json")

In [6]:
import numpy as np
import torch
def get_avg_probability_hf(logits):
    num_produced_tokens = len(logits) - 1 #ignore <\s> token at the end of the generation
    sum_linear_probs = []

    for i in range(num_produced_tokens):
        probabilities = torch.log_softmax(logits[i], dim=-1).cpu()
        top_logprobs, _ = torch.topk(probabilities, 3)
        linear_probability_top_token = np.exp(top_logprobs[0][0])
        sum_linear_probs.append(linear_probability_top_token)
    return np.mean(sum_linear_probs).item()

def get_perplexity_hf(logits):
    num_produced_tokens = len(logits) - 1 #ignore <\s> token at the end of the generation
    nll = []
    for i in range(num_produced_tokens):
        probabilities = torch.log_softmax(logits[i][0], dim=-1).cpu()
        top_logprobs, _ = torch.topk(probabilities, 3)
        top_logprob = top_logprobs[0]
        nll.append(top_logprob.cpu())
    avg_nll = np.mean(nll)
    ppl = np.exp(-avg_nll)
    return ppl.item()
def get_avg_entropy_hf(logits):
    k = 10
    num_produced_tokens = len(logits) - 1 #ignore <\s> token at the end of the generation
    sum_all_entropies = 0

    for i in range(num_produced_tokens):
        entropy_current_position = 0
        probabilities = torch.log_softmax(logits[i], dim=-1).cpu()
        top_logprobs, _ = torch.topk(probabilities, k)

        for logprob in top_logprobs[0]:
            linear_probability = np.exp(logprob)
            if torch.isinf(logprob):
                logprob = torch.tensor(0)
            entropy_current_position += linear_probability * logprob

        sum_all_entropies += -(entropy_current_position)
    answer_entropy = sum_all_entropies / num_produced_tokens
    return answer_entropy.item()

In [ ]:
 from collections import defaultdict
 allans_basemodel=[]
 correct=0
 for idx, sample in dataset.iterrows():
    print(idx)
    correct_gen_ans = False
    question = sample["question"]
    true_answer = sample["true_answer"]
    false_info_context = sample["context"]
    questionvals=question+" Respond with the exact answer only."
    logits,model_answer=getAnswer_completion(questionvals)

    answer_entropy = get_avg_entropy_hf(logits)
    answer_perplexity = get_perplexity_hf(logits)
    answer_probability = get_avg_probability_hf(logits)

    uncertainty_results = defaultdict(list)
    uncertainty_results["avg_entropy"].append(answer_entropy)
    uncertainty_results["avg_perplexity"].append(answer_perplexity)
    uncertainty_results["avg_probability"].append(answer_probability)
    if true_answer.lower() in model_answer.lower():
        correct_gen_ans=True
        correct += 1
    entry = {
    "id": idx,
    "question": question,
    "true_answer": true_answer,
    "model_baseprompt_answer": model_answer,
    "correct_gen_ans": correct_gen_ans,
    "false_context": false_info_context,
    "uncertainty":uncertainty_results
    }
    allans_basemodel.append(entry)

In [8]:
allans=pd.DataFrame(allans_basemodel)
allans.to_json("../outputs/safe_finetuned_model/safe_finetuned_correct_baseprompt_results.json")

In [9]:
acc = (len(allans[allans['correct_gen_ans']==True]) / len(dataset)) * 100
print(f"Accuracy: {acc}%")

Accuracy: 55.50000000000001%


## Generate answers with false context

In [ ]:
from collections import defaultdict
allans_fc_model=[]
correct=0
for idx, sample in dataset.iterrows():
    print(idx)
    correct_gen_ans = False
    question = sample["question"]
    true_answer = sample["true_answer"]
    false_info_context = sample["context"]
    questionvals=question+" Respond with the exact answer only."
    falseinfo_prompt= false_info_context+questionvals
    logits,model_answer=getAnswer_completion(falseinfo_prompt)

    answer_entropy = get_avg_entropy_hf(logits)
    answer_perplexity = get_perplexity_hf(logits)
    answer_probability = get_avg_probability_hf(logits)

    uncertainty_results = defaultdict(list)
    uncertainty_results["avg_entropy"].append(answer_entropy)
    uncertainty_results["avg_perplexity"].append(answer_perplexity)
    uncertainty_results["avg_probability"].append(answer_probability)
    if true_answer.lower() in model_answer.lower():
        correct_gen_ans= True
        correct += 1
    entry = {
    "id": idx,
    "question": question,
    "true_answer": true_answer,
    "model_falsecontext_answer": model_answer,
    "false_context": false_info_context,
    "correct_gen_ans": correct_gen_ans,
    "uncertainty":uncertainty_results
    }
    allans_fc_model.append(entry)

In [11]:
allAns=pd.DataFrame(allans_fc_model)
allAns.to_json("../outputs/safe_finetuned_model/safe_finetuned_false_context_results.json")

In [12]:
acc = (len(allAns[allAns['correct_gen_ans']==True]) / len(dataset)) * 100
print(f"Accuracy: {acc}%")

Accuracy: 44.4%


In [ ]:
from collections import defaultdict
import random
allans_rc_model=[]
correct=0
contexts = dataset["context"].tolist()
random.seed(42)
random.shuffle(contexts)
for idx, sample in dataset.iterrows():
    print(idx)
    correct_gen_ans = False
    question = sample["question"]
    true_answer = sample["true_answer"]
    random_context =  contexts[idx]
    questionvals=question+" Respond with the exact answer only."
    falseinfo_prompt= random_context+questionvals
    logits,model_answer=getAnswer_completion(falseinfo_prompt)

    answer_entropy = get_avg_entropy_hf(logits)
    answer_perplexity = get_perplexity_hf(logits)
    answer_probability = get_avg_probability_hf(logits)

    uncertainty_results = defaultdict(list)
    uncertainty_results["avg_entropy"].append(answer_entropy)
    uncertainty_results["avg_perplexity"].append(answer_perplexity)
    uncertainty_results["avg_probability"].append(answer_probability)
    if true_answer.lower() in model_answer.lower():
        correct_gen_ans= True
        correct += 1
    entry = {
    "id": idx,
    "question": question,
    "true_answer": true_answer,
    "model_randomcontext_answer": model_answer,
    "random_context": random_context,
    "correct_gen_ans": correct_gen_ans,
    "uncertainty":uncertainty_results
    }
    allans_rc_model.append(entry)

In [14]:
allAns=pd.DataFrame(allans_rc_model)
allAns.to_json("../outputs/safe_finetuned_model/safe_finetuned_random_context_results.json")

In [15]:
acc = (len(allAns[allAns['correct_gen_ans']==True]) / len(dataset)) * 100
print(f"Accuracy: {acc}%")

Accuracy: 45.2%
